In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [100]:
#change path
path="..\\UB_keystroke_dataset\\UB_keystroke_dataset"

In [101]:
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))



In [120]:
len(files)
print(files[0:1])

['..\\UB_keystroke_dataset\\UB_keystroke_dataset\\s0\\baseline\\001000.txt']


In [113]:
items = []
for file in files[0:1]:
    f=open(file, "r")
    for line in f:
        words= line.rstrip('\n').split(sep=" ")
        items.append((words[0],words[1],words[2]))


In [114]:
df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time'])
print (df.shape)

(7792, 3)


In [115]:
temp=df[df["Key"].str.match('^.*[A-Z]$')]

In [116]:
temp.groupby("Key").size()/2

Key
A    225.0
B     44.0
C     65.0
D    138.0
E    323.0
F     61.0
G     63.0
H    119.0
I    183.0
J      2.0
K     18.0
L    128.0
M     67.0
N    159.0
O    228.0
P     53.0
Q      2.0
R    150.0
S    161.0
T    296.0
U     81.0
V     27.0
W     50.0
X      5.0
Y     71.0
dtype: float64

In [107]:
temp

,Key,EventType,Time
47,I,KeyDown,63578429792602
48,I,KeyUp,63578429792696
52,A,KeyDown,63578429792961
53,A,KeyUp,63578429793054
54,M,KeyDown,63578429793257
55,M,KeyUp,63578429793382
58,H,KeyDown,63578429793616
59,H,KeyUp,63578429793741
60,O,KeyDown,63578429793756
61,O,KeyUp,63578429793881


In [117]:
prev=0
rows=[]
for index,row in temp.iterrows():
    if prev==0 and row[1]=="KeyDown":
        prevRow=row
        prev=1
    elif row[1]=="KeyDown":
        rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2])])
        prevRow=row

In [118]:
len(rows)

2718

In [119]:
g = mixture.GMM(n_components=2)

In [ ]:
testPath='..\\UB_keystroke_dataset\\UB_keystroke_dataset\\s1\\baseline\\001000.txt'